###Classification

In [ ]:
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten,GlobalAveragePooling2D,MaxPool2D,Conv2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import Model
from keras.layers import BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras import layers
from keras.preprocessing import image
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
def predict_imageclassify(model, image):
    predict = model.predict(image, verbose=1)
    return predict[0]

In [ ]:
def classification(classified_imgfile):
  image= Image.open(classified_imgfile)
  img, h, w = load_data(classified_imgfile)
  modelclassify= tf.keras.models.load_model('/content/classification.h5')
  prediction = modelclassify.predict(img)
  
  return prediction

###Segmentation(Task_2)

In [ ]:
from tensorflow import keras
import tensorflow as tf
import os
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten,GlobalAveragePooling2D, Lambda, Conv2DTranspose, concatenate
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import Model
from keras.layers import BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras import layers
from keras.preprocessing import image
import seaborn as sns
import zipfile
import shutil
import cv2
from PIL import Image

In [ ]:
def load_data(INPUT_FILE):
    
         img = cv2.imread(INPUT_FILE, cv2.IMREAD_GRAYSCALE)
         img = img/255.0
         height, width = img.shape[:2]
         img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
         img = img.reshape(1, 256, 256, 1)                                             
         return img, height, width

In [ ]:
def predict_image(model, image):
    predict = model.predict(image, verbose=1)
    return predict[0] 

In [ ]:

def segmentation(classified_imgfile):
  image= Image.open(classified_imgfile)
  img, h, w = load_data(classified_imgfile)
  modelsegment= tf.keras.models.load_model('/content/segmentation.h5')

  output_image = predict_image(modelsegment, classified_imgfile )
  return output_image

###Deskewing

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from skimage.color import rgb2gray
from skimage.transform import rotate
from skimage.transform import (hough_line, hough_line_peaks)
from scipy.stats import mode
from skimage import io
from skimage.filters import threshold_otsu, sobel
from matplotlib import cm


In [ ]:
def binarize_image(rgb_image):
    image = rgb2gray(rgb_image)
    threshold = threshold_otsu(image)
    binary_image = image < threshold

    return binary_image

In [ ]:
def find_edges(binary_image):
    image_edges = sobel(binary_image)

    plt.imshow(binary_image, cmap='gray')
    plt.axis('off')
    plt.title('Binary Image Edges')
    plt.savefig('/content/drive/My Drive/binary_image.png')

    return image_edges

In [ ]:
def find_tilt_angle(image_edges):
    h, theta, d = hough_line(image_edges)
    accum, angles, dists = hough_line_peaks(h, theta, d)
    angle = np.rad2deg(mode(angles)[0][0])

    if angle < 0:
        r_angle = angle + 90
    else:
        r_angle = angle - 90

    # Plot Image and Lines
    fig, ax = plt.subplots()

    ax.imshow(image_edges, cmap='gray')

    origin = np.array((0, image_edges.shape[1]))

    for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
        y0, y1 = (dist - origin * np.cos(angle)) / np.sin(angle)
        ax.plot(origin, (y0, y1), '-r')

    ax.set_xlim(origin)
    ax.set_ylim((image_edges.shape[0], 0))
    ax.set_axis_off()
    ax.set_title('Detected lines')

    plt.savefig('/content/drive/My Drive/hough_lines.png')
    plt.show()

    return r_angle


In [ ]:
def rotate_image(rgb_image, angle):
    fixed_image = rotate(rgb_image, angle)

    plt.imshow(fixed_image)
    plt.axis('off')
    plt.title('Fixed Image')
    plt.savefig('/content/drive/My Drive/fixed_image.png')
    plt.show()

    return fixed_image


def deskew_image(img_path):
    image = io.imread(img_path)
    binary_image = binarize_image(image)
    image_edges = find_edges(binary_image)
    angle = find_tilt_angle(image_edges)
    rotate_image(io.imread(img_path), angle)


###OCR

In [ ]:
pip install pytesseract
!sudo apt install tesseract-ocr

In [ ]:
import pytesseract
from PIL import Image

def extract_text(image_path):
    # Load the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    image = image.convert("L")

    # Apply OCR using Tesseract
    words = pytesseract.image_to_string(image).split()

    # Return the recognized words
    return words

# Test the function with an image
image_path = "/content/drive/MyDrive/IPDA dataset/ipda22_datasets/5_ocr/fake_id_11__front.png"
words = extract_text(image_path)
print(words)

